# Setup

Dependencies:
- System: python3
- Python: jupyter, matplotlib, numpy, scipy

Example setup for a Ubuntu system (Mac users, maybe `brew` instead of `sudo apt`; Windows users, learn to love [WSL](https://docs.microsoft.com/en-us/windows/wsl/install-win10)):
```
/usr/bin/python3 -m pip install --upgrade pip
pip install --upgrade jupyter matplotlib numpy scipy
jupyter nbextension enable --py widgetsnbextension  # necessary for interactive sliders to show up
jupyter notebook  # from the directory of this notebook
```
Alternatively, view this notebook on [Google Colab](https://colab.research.google.com/github/StanfordASL/AA203-Examples/blob/master/Lecture-2/Model%20Reference%20Adaptive%20Control.ipynb).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

from ipywidgets import interact

In [ ]:
def step(t):
    return np.ones_like(t)


def sinusoidal(t):
    return np.sin(t / 3)


def simulate_mrac(m_hat_0=0.5, reference_signal=step, time_horizon=20.):
    # True mass.
    m = 1.

    # Ideal response gains.
    k_d = k_p = k = 1.

    # Adaptation gains.
    λ = 1.
    γ = 1.

    def simulation_ode(t, state_refstate_mhat):
        x, dx, x_m, dx_m, m_hat = state_refstate_mhat
        ddx_m = k * reference_signal(t) - k_d * dx_m - k_p * x_m
        s = (dx - dx_m) + λ * (x - x_m)
        ν = ddx_m - 2 * λ * (dx - dx_m) - λ**2 * (x - x_m)
        u = m_hat * ν
        return np.array([dx, u / m, dx_m, ddx_m, -γ * ν * s])

    sol = solve_ivp(simulation_ode, (0., time_horizon),
                    np.array([0., 0., 0., 0., m_hat_0]),
                    t_eval=np.linspace(0, time_horizon, 400))

    plt.figure(figsize=(20, 10))
    plt.plot(sol.t, sol.y.T, label=["x", "ẋ", "xₘ", "ẋₘ", "m̂"])
    plt.plot(sol.t, reference_signal(sol.t), "--", label="reference")

    x, dx, x_m, dx_m, m_hat = sol.y
    s = (dx - dx_m) + λ * (x - x_m)
    plt.plot(sol.t, (m * s**2 + (m - m_hat)**2 / γ) / 2, "--", label="V")
    plt.plot(sol.t, np.hypot(x - x_m, dx - dx_m), "--", label="state_error")
    plt.legend(fontsize=20)

In [ ]:
interact(simulate_mrac, m_hat_0=(0.1, 2.0), reference_signal=[step, sinusoidal], time_horizon=[20, 100, 1000])